# Crear el archivo de entrada con datos meteorológicos

En este cuaderno aprenderás a crear datos de entrada para el modelo glaciológico. Usaremos datos meteorológicos y topográficos. Usaremos un método simple de interpolación para distribuir los datos en todo el glaciar. 

## Importar librerias **`xarray`** y **`numpy`**

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

## Leer datos

In [ ]:
# Topográficos
ds_met = xr.open_dataset('./data/ERA5_bell.nc')
# Meteorológicos
ds_sta = xr.open_dataset('../SES6/dom/Bell_dom.nc')

In [ ]:
ds_met

In [ ]:
ds_sta

## Hacer una copia para guardar los datos de entrada

In [ ]:
dso = ds_sta.copy()    
dso.coords['time'] = ds_met.valid_time.values
dso

## Interpolar las variables de temperatura y precipitación 

In [ ]:
T_interp   = np.zeros([len(dso.time), len(ds_sta.lat), len(ds_sta.lon)])
RRR_interp = np.zeros([len(dso.time), len(ds_sta.lat), len(ds_sta.lon)])

In [ ]:
lapse_T   = -6.5/1000 # °C / m
lapse_RRR =  (0.10/100) # % / m
g = 9.81
ERA5Alt = (ds_met.z[0,0,0].values)/g
T2  = ds_met.t2m[:,0,0].values
RRR = ds_met.tp[:,0,0].values*1000
print(ERA5Alt)

In [ ]:
plt.plot(RRR)

In [ ]:
for t in range(len(dso.time)):
    T_interp[t,:,:] = (T2[t]) + (ds_sta.HGT.values-ERA5Alt)*lapse_T

    if RRR[t] <= 0.20:

        RRR_interp[t,:,:] = 0.0
    else:
        RRR_interp[t,:,:] = (RRR[t]) + (ds_sta.HGT.values-ERA5Alt)*lapse_RRR

RRR_interp[RRR_interp < 0] = 0



## Crear una función para guardar los datos interpolados

In [ ]:

def add_variable_along_timelatlon(ds, var, name, units, long_name):
    """ This function adds missing variables to the DATA class """
    ds[name] = (('time','lat','lon'), var)
    ds[name].attrs['units'] = units
    ds[name].attrs['long_name'] = long_name
    return ds

## Adicionar las variables en el netcdf

In [ ]:
add_variable_along_timelatlon(dso, T_interp, 'T2', 'K', 'Temperature at 2 m')
add_variable_along_timelatlon(dso, RRR_interp, 'RRR', 'mm', 'Total precipitation (liquid+solid)')

## Visualizar los datos

In [ ]:
dso.T2[0,:,:].plot()

In [ ]:
dso.RRR.max()

In [ ]:
dso.RRR[30,:,:].plot()

## Guardar el netcdf

In [ ]:
dso.to_netcdf('./data/ERA5_input_bell.nc')